In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random

In [60]:
root_path = '/content/drive/My Drive/data'
for item in os.listdir(root_path):
    item_path = os.path.join(root_path, item)
    if os.path.isdir(item_path):
        print(f'Папка: {item}')
    else:
        print(f'Файл: {item}')

Папка: running
Папка: walking
Папка: idle
Папка: stairs


In [64]:
activity_mapping = {
    'running': 'running',
    'walking': 'walking',
    'idle': 'idle',
    'stairs': 'stairs'
}

df_list = []
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        folder_df_list = []
        activity = activity_mapping[folder_name]
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                df = df.assign(activities=activity)
                folder_df_list.append(df)
        if folder_df_list:
            combined_folder_df = pd.concat(folder_df_list)
            df_list.append(combined_folder_df)

combined_df = pd.concat(df_list)
print(combined_df.head())

   accelerometer_X  accelerometer_Y  accelerometer_Z activities
0       -22.998701        31.598675       -26.297910    running
1        -5.942408         2.935291       -11.113884    running
2         7.886501        14.863204        -8.508993    running
3       -11.836933         1.216255        -8.456321    running
4       -19.306843       -13.919888         3.581726    running


In [66]:
X = combined_df[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
y = combined_df['activities']

In [67]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [69]:
svm_model = SVC()
rf_model = RandomForestClassifier()

svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

svm_preds = svm_model.predict(X_test)
rf_preds = rf_model.predict(X_test)

print("Classification Report for SVM:")
print(classification_report(y_test, svm_preds))

print("Classification Report for Random Forest:")
print(classification_report(y_test, rf_preds))

Classification Report for SVM:
              precision    recall  f1-score   support

        idle       0.96      0.98      0.97      6221
     running       0.93      0.90      0.92     20510
      stairs       1.00      0.00      0.01       915
     walking       0.80      0.90      0.85     11126

    accuracy                           0.89     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.89      0.88     38772

Classification Report for Random Forest:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6221
     running       1.00      1.00      1.00     20510
      stairs       1.00      1.00      1.00       915
     walking       1.00      1.00      1.00     11126

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



In [71]:
svm_report = classification_report(y_test, svm_preds, output_dict=True)
rf_report = classification_report(y_test, rf_preds, output_dict=True)

In [72]:
svm_precision = svm_report['macro avg']['precision']
rf_precision = rf_report['macro avg']['precision']

svm_recall = svm_report['macro avg']['recall']
rf_recall = rf_report['macro avg']['recall']

svm_f1 = svm_report['macro avg']['f1-score']
rf_f1 = rf_report['macro avg']['f1-score']

print("SVM Precision:", svm_precision)
print("Random Forest Precision:", rf_precision)

print("SVM Recall:", svm_recall)
print("Random Forest Recall:", rf_recall)

print("SVM F1-score:", svm_f1)
print("Random Forest F1-score:", rf_f1)

SVM Precision: 0.922553988520022
Random Forest Precision: 0.9999232560310127
SVM Recall: 0.6973385707592235
Random Forest Recall: 0.9994188928462097
SVM F1-score: 0.685356853905337
Random Forest F1-score: 0.9996707727927276
